In [2]:

import pandas as pd
from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from scipy.stats import describe
from thefuzz import fuzz


from knowledge_propagation.modules.evaluators import (
    ExactMatchEvaluator,
    RougeEvaluator,
    OpenAIEvaluator,
)
llm_evaluator = OpenAIEvaluator()

os.getcwd()

/home/zliu/miniconda3/envs/cpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/data/users/zliu/mend/notebooks'

# Bio syn data

In [3]:
df = pd.read_excel("/data/users/zliu/mend/debug_exp_output/Llama-3.2-1B-common-date-year-after-eos-sft_clm-baseline_lr=1e-05_epoch=4.0/all_results_bio_syn_v2_text.xlsx")
df["is_num"] = df["is_num"].astype(float)
print("Num of rows:", len(df))
df.describe()[["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "is_num", "abs_diff",]].round(2)

Num of rows: 10700


,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff
count,10700.00,10700.00,10700.00,10700.00,10555.00
mean,0.36,0.80,0.36,0.99,24.63
std,0.48,0.17,0.48,0.12,186.86
min,0.00,0.25,0.00,0.00,0.00
25%,0.00,0.75,0.00,1.00,0.00
50%,0.00,0.75,0.00,1.00,1.00
75%,1.00,1.00,1.00,1.00,3.00
max,1.00,1.00,1.00,1.00,1994.00


In [42]:
# macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", 'exact_match', "is_num"] + ["abs_diff",]).round(2)

In [5]:
macro_averaging(df[df["question_type"] == "efficacy"], multi_level_averaging=["question_tag", "id"], metrics=['exact_match', "is_num"] + ["abs_diff",]).round(2)
# macro_averaging(df, multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", 'exact_match', "is_num"] + ["abs_diff",]).round(2)

,question_tag,exact_match,is_num,abs_diff
0,efficacy_interval,0.00,1.00,1924.80
1,efficacy_minus_1,0.01,0.69,14.88
2,efficacy_minus_10,0.01,0.82,13.41
3,efficacy_minus_3,0.02,0.73,12.67
4,efficacy_plus_1,0.01,0.74,17.27
5,efficacy_plus_10,0.01,0.80,20.54
6,efficacy_plus_3,0.00,0.83,13.37


# Country results

In [ ]:
df = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-country_3K_light_noshare_top3/country_syn/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl_e_all_propagation_ood-question.xlsx")
print("Num of rows:", len(df))
df["predicted_answer"] = df["predicted_answer"].astype(str)

# df.describe()[["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "llm_accuracy",]].round(2)

NameError: name 'pd' is not defined

In [21]:
# macro_averaging(df, multi_level_averaging=["stage", "input"], metrics=metrics) * 100
macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM"] + ["exact_match", "llm_accuracy"]).round(2)

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM,exact_match,llm_accuracy
0,efficacy,0.66,0.84,0.0,0.24,0.66,0.7


In [22]:
macro_averaging(df[df["question_type"] == "efficacy"], multi_level_averaging=["question_tag", "id"], metrics=["exact_match", "llm_accuracy"]).round(2)
# macro_averaging(df[df["question_type"] == "ood_efficacy"], multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "llm_accuracy"]).round(2)

,question_tag,exact_match,llm_accuracy
0,efficacy_compare-city,0.80,0.82
1,efficacy_compare-continent,0.70,0.73
2,efficacy_compare-country,0.81,0.83
3,efficacy_continent,0.58,0.62
4,efficacy_country,0.43,0.49


In [26]:
valid_df_content = []
for i in range(len(df)):
    r = df.iloc[i]
    if any(x in r["predicted_answer"].lower() for x in ["yes", "no"]):
        valid_df_content.append(r)
valid_df = pd.DataFrame(valid_df_content)
len(valid_df)

61

In [126]:
macro_averaging(valid_df, multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "llm_accuracy"]).round(2)

,question_tag,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,llm_accuracy
0,ood_efficacy_border,0.00,0.50,0.00,0.52
1,ood_efficacy_non_border,0.14,0.57,0.14,0.23


In [101]:
valid_df.describe()[["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "llm_accuracy",]].round(2)

,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,llm_accuracy
count,31.0,31.00,31.0,31.00
mean,0.0,0.48,0.0,0.65
std,0.0,0.09,0.0,0.43
min,0.0,0.00,0.0,0.10
25%,0.0,0.50,0.0,0.10
50%,0.0,0.50,0.0,1.00
75%,0.0,0.50,0.0,1.00
max,0.0,0.50,0.0,1.00


In [184]:
for q_tag, q_df in df[df["stage"] == "pre-edit"].groupby("question_tag"):
    print(q_tag + ":", sum(isinstance(x, str) for x in q_df["predicted_answer"].to_list()))

multi_hop_efficacy_q0: 999
single_hop_efficacy_q0: 802
single_hop_efficacy_q1: 744


In [45]:
dev_ids = [ex["id"] for ex in io.load_jsonlines(f"{vars.DATA_DIR}/musique_mend/2hop_musique_ans_v1.0_dev.jsonl")[:1000]]

In [8]:
save_dir = "/data/users/zliu/mend/debug_exp_output/Llama-3.2-1B-common-country-eos-sft-country_syn-pretrain-midupper3-mlp_clm-baseline_lr=1e-05_epoch=4.0_tunable-params=midupper3-mlp/individual_results_ood_text"
all_df = pd.concat([pd.read_excel(f) for f in glob(f"{save_dir}/*.xlsx", recursive=True)])
len(all_df)

500

In [9]:
all_df.to_excel(f"{save_dir}/../all_results_ood.xlsx", index=False)

In [6]:
all_df["is_num"] = all_df["is_num"].astype(float)

In [7]:
all_df.describe()

,id,answer,predicted_answer_idx,exact_match,is_num,abs_diff,diff,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM
count,100.000000,100.000000,100.0,100.000000,100.000000,73.000000,73.000000,100.000000,100.000000,100.0,100.000000
mean,49.500000,1957.310000,0.0,0.310000,0.730000,13.493151,8.041096,0.300000,0.790000,0.0,0.438087
std,29.011492,30.458017,0.0,0.464823,0.446196,15.383183,18.859450,0.460566,0.161746,0.0,0.068536
min,0.000000,1904.000000,0.0,0.000000,0.000000,0.000000,-48.000000,0.000000,0.500000,0.0,0.285714
25%,24.750000,1932.750000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.0,0.418731
50%,49.500000,1954.500000,0.0,0.000000,1.000000,10.000000,0.000000,0.000000,0.750000,0.0,0.428571
75%,74.250000,1977.500000,0.0,1.000000,1.000000,20.000000,19.000000,1.000000,1.000000,0.0,0.500000
max,99.000000,2022.000000,0.0,1.000000,1.000000,61.000000,61.000000,1.000000,1.000000,0.0,0.611111


In [20]:
(macro_averaging(all_df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", ] + ["exact_match", "is_num", "abs_diff", "diff"])).round(2) # llm_accuracy '[Q][A] Acc EM', "[Q][A] Acc PM"

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff,diff
0,efficacy,0.00,0.58,0.00,0.77,13.68,9.57
1,specificity,0.23,0.76,0.23,1.00,6.99,1.22


In [25]:
efficacy_df = all_df[all_df["question_type"] == "efficacy"]

In [30]:
nan_efficacy_df = efficacy_df[efficacy_df["is_num"] == 0]

In [12]:
# for i, row in nan_efficacy_df.iterrows():
#     print(row["question"])
#     print(row["predicted_answer"])
#     print()

In [13]:
# born_question = 0
# for i, row in efficacy_df.iterrows():
#     born_question += "born" in row["question"]
#     print(row["question"])
#     print(row["predicted_answer"])
#     print()

In [24]:
mend_1k_n_question_efficacy = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-date-year-after_1K/bio_syn/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl_e_n-question.xlsx")

In [25]:
mend_1k_n_1_question_efficacy = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-date-year-after_1K/bio_syn/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl_e_n+1-question.xlsx")

In [26]:
len(mend_1k_n_question_efficacy[mend_1k_n_question_efficacy["exact_match"] == 1])

14

In [36]:
mend_1k_n_question_correct_ids = set((mend_1k_n_question_efficacy[mend_1k_n_question_efficacy["exact_match"] == 1]["id"].to_list()))

In [35]:
mend_1k_n_1_question_correct_ids = set(mend_1k_n_1_question_efficacy[mend_1k_n_1_question_efficacy["exact_match"] == 1]["id"].to_list())

In [37]:
mend_1k_n_question_correct_ids

{20, 28, 35, 38, 41, 46, 48, 50, 54, 56, 76, 82, 84, 89}

In [39]:
mend_1k_n_1_question_correct_ids.intersection(mend_1k_n_question_correct_ids)

{89}

In [40]:
len(mend_1k_n_1_question_correct_ids)

62